In [1]:
import os, random, sys, time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
import nltk

In [2]:
driver = webdriver.Chrome()
file = open('login.txt')
lines = file.readlines()
username =lines[0]
password =lines[1]
driver.get("https://www.linkedin.com/uas/login")
element= driver.find_element_by_id('username')
element.send_keys(username)
element =driver.find_element_by_id('password')
element.send_keys(password)
element.submit()
driver.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')

In [3]:
profilelist=[]
visitprofile =[]

In [6]:
def profile(soup,profilelist):
    profile=[]
    results =soup.find('div',{"class":'scaffold-finite-scroll__content'})
    all_result=results.findAll('a',{'class':'ember-view mn-connection-card__picture'})
    for result in all_result:
        user=result.get('href') 
        profile.append(user)
    return profile

In [7]:
profile(BeautifulSoup(driver.page_source),profilelist)
profilelist= profile(BeautifulSoup(driver.page_source),profilelist)
len(profilelist)

943

In [8]:
records=[]

In [50]:
while profilelist:
    visiting = profilelist.pop()
    visitprofile.append(visiting)
    link = 'https://www.linkedin.com' + visiting
    driver.get(link)
    time.sleep(15)
    soup=driver.page_source
    result=BeautifulSoup(soup,'lxml')
    def name():
        results =result.find('h1',{"class":'text-heading-xlarge inline t-24 v-align-middle break-words'})
        ali=results.text.strip()
        return ali
    def title():
        results =result.find('div',{"class":'text-body-medium break-words'})
        ali2=results.text.strip()
        return ali2
    def place():
        results =result.find('span',{"class":'text-body-small inline t-black--light break-words'})
        ali3=results.text.strip()
        return ali3
    def posts():
        prepost=[]
        filtered_sentence = []
        results =result.find('ul',{"class":'pv-profile-section__section-info section-info pv-profile-section__section-info--has-no-more'})
        ali=results.findAll('h3',{'class':'t-16 t-black t-bold'})
        for post in ali:
            prepost.append(post.text)
        filtered_sentence = (" ").join(prepost)    
        return filtered_sentence
    def experience():
        noofexprience=[]
        filtered_sentence = []
        results =result.find('ul',{"class":'pv-profile-section__section-info section-info pv-profile-section__section-info--has-no-more'})
        ali=results.findAll('span',{'class':'pv-entity__bullet-item-v2'})
        for post in ali:
            noofexprience.append(post.text.strip())
        filtered_sentence = (" ").join(noofexprience)    
        return filtered_sentence
    def educationname():
        schoolname=[]
        filtered_sentence = []
        ali=result.findAll('h3',{'class':'pv-entity__school-name t-16 t-black t-bold'})
        for post in ali:
            schoolname.append(post.text.strip())
        filtered_sentence = (" ").join(schoolname)    
        return filtered_sentence
        
    def degreename():
        degreename=[]
        filtered_sentence = []
        ali=result.findAll('span',{'class':'pv-entity__comma-item'})
        for post in ali:
            degreename.append(post.text.strip())
        filtered_sentence = (" ").join(degreename)    
        return filtered_sentence
    
    def responsibility():
        responsibility=[]
        filtered_sentence = []
        results =result.find('ul',{"class":'pv-profile-section__section-info section-info pv-profile-section__section-info--has-no-more'})
        ali=results.findAll('div',{'class':'pv-entity__description t-14 t-black t-normal inline-show-more-text inline-show-more-text--is-collapsed ember-view'})
        for post in ali:
            responsibility.append(post.text.strip())
            punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
            no_punct = ""
            for char in post.text:
                if char not in punctuations:
                    no_punct = no_punct + char
                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(no_punct)  
                    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
                    for w in word_tokens:  
                        if w not in stop_words:  
                            filtered_sentence.append(w)  
            filtered_sentence = (" ").join(word_tokens)
            return filtered_sentence
    def skills():
        skills=[]
        filtered_sentence = []
        ali=result.findAll('p',{'class':'pv-skill-category-entity__name pv-skill-category-entity__name--has-action tooltip-container'})
        for post in ali:
            skills.append(post.text.strip())
        filtered_sentence = (" ").join(skills)
        return filtered_sentence
    records.append((name(),title(),place(),posts(),experience(),responsibility(),educationname(),degreename(),skills()))
    df = pd.DataFrame(records, columns=['NAME','TITLE','PLACE','POST','EXPRIENCE','RESPONSIBILITY','EDUCATION','DEGREE','SKILLS'])
    
    

AttributeError: 'NoneType' object has no attribute 'text'

In [51]:
df

,NAME,TITLE,PLACE,POST,EXPRIENCE,RESPONSIBILITY,EDUCATION,DEGREE,SKILLS
0,ashir khan,Attended muhammad ali jinnah university,Pakistan,,,None,muhammad ali jinnah university,,
1,Saqib Ali,Manager Agent Network (Digital Banking) at Ban...,"Karāchi, Sindh, Pakistan",,,None,Imperial College of Business Studies Shaheed Z...,Master of Management (Management Sciences) Pro...,Sales Operations Management Team Management Sa...
2,Umaid Mustafa,Chief Executive Officer at Re-Marketing Solutions,"Karāchi, Sindh, Pakistan",Chief Executive Officer Web Development & Mark...,3 mos 1 yr 2 mos 11 mos 6 mos 3 mos 3 mos,None,Muhammad Ali Jinnah University,Computer science Computer Science,Social Media Leadership Project Management Dat...
3,Rameez (Ramiz) Ul Haq Ansari,Software Engineer at Careem,Pakistan,Software Engineer Full Stack Developer Social ...,1 yr 6 mos 4 mos 2 mos,None,Usman Institute of Technology DJ Science College,Bachelor of Science (BS) Computer Software Eng...,C# Microsoft Office C++ Graphic Design Android...
4,Dr. Shaukat Wasi,"Associate Dean, Faculty of Computing at Mohamm...","Karāchi, Sindh, Pakistan",\nCompany Name\nMohammad Ali Jinnah University...,1 yr 6 mos 5 yrs 2 mos 3 yrs 4 yrs 5 yrs 6 mos...,None,National University of Computer and Emerging S...,PhD Computer Science A MS Computer Science BS ...,C++ Information Retrieval Text Mining Computer...
...,...,...,...,...,...,...,...,...,...
255,Talha Shahid,Frontend Developer at KNYSYS LLC,"Karāchi, Sindh, Pakistan",Frontend Developer Web Developer Web Developer...,4 mos 10 mos 1 yr 1 mo 1 yr 4 mos,None,Mohammad ali jinnah university,Bachelor of Science - BS Computer Science,C# .NET Framework Microsoft Office Web Develop...
256,BILAL SHAH KHAN,Logistics Supply Chain Specialist at TRAX,Pakistan,Logistics Supply Chain Specialist \nCompany Na...,11 mos 10 mos 8 mos 3 yrs 1 yr,None,Karachi University APTECH Computer Education g...,"Bachelor of Commerce (BCom) Business/Commerce,...",Teamwork Microsoft Excel Sales Marketing Sales...
257,Ahsan Shafiq,Sr. UI Engineer at Folio3 Software,"Karachi Division, Pakistan",,No Expiration Date No Expiration Date,None,Muhammad Ali Jinnah University Aptech Computer...,Software Engineering Information Technology BS...,jQuery CSS JavaScript Web Development Mobile A...
258,Hammad Masood,CEO| Co-founder at Doozie Labs,Pakistan,Co-founder Software Engineer,8 yrs 2 mos 1 yr 1 mo,None,National University of Computer and Emerging S...,Master of Science (MS) Computer Science 3.11 C...,MySQL Android C++ Android Development OOP Prog...


In [52]:
df.to_csv("Clean.csv", header=True, index=True)

In [53]:
df = pd.read_csv('Clean.csv')
df

,Unnamed: 0,NAME,TITLE,PLACE,POST,EXPRIENCE,RESPONSIBILITY,EDUCATION,DEGREE,SKILLS
0,0,ashir khan,Attended muhammad ali jinnah university,Pakistan,NaN,NaN,NaN,muhammad ali jinnah university,NaN,NaN
1,1,Saqib Ali,Manager Agent Network (Digital Banking) at Ban...,"Karāchi, Sindh, Pakistan",NaN,NaN,NaN,Imperial College of Business Studies Shaheed Z...,Master of Management (Management Sciences) Pro...,Sales Operations Management Team Management Sa...
2,2,Umaid Mustafa,Chief Executive Officer at Re-Marketing Solutions,"Karāchi, Sindh, Pakistan",Chief Executive Officer Web Development & Mark...,3 mos 1 yr 2 mos 11 mos 6 mos 3 mos 3 mos,NaN,Muhammad Ali Jinnah University,Computer science Computer Science,Social Media Leadership Project Management Dat...
3,3,Rameez (Ramiz) Ul Haq Ansari,Software Engineer at Careem,Pakistan,Software Engineer Full Stack Developer Social ...,1 yr 6 mos 4 mos 2 mos,NaN,Usman Institute of Technology DJ Science College,Bachelor of Science (BS) Computer Software Eng...,C# Microsoft Office C++ Graphic Design Android...
4,4,Dr. Shaukat Wasi,"Associate Dean, Faculty of Computing at Mohamm...","Karāchi, Sindh, Pakistan",\nCompany Name\nMohammad Ali Jinnah University...,1 yr 6 mos 5 yrs 2 mos 3 yrs 4 yrs 5 yrs 6 mos...,NaN,National University of Computer and Emerging S...,PhD Computer Science A MS Computer Science BS ...,C++ Information Retrieval Text Mining Computer...
...,...,...,...,...,...,...,...,...,...,...
255,255,Talha Shahid,Frontend Developer at KNYSYS LLC,"Karāchi, Sindh, Pakistan",Frontend Developer Web Developer Web Developer...,4 mos 10 mos 1 yr 1 mo 1 yr 4 mos,NaN,Mohammad ali jinnah university,Bachelor of Science - BS Computer Science,C# .NET Framework Microsoft Office Web Develop...
256,256,BILAL SHAH KHAN,Logistics Supply Chain Specialist at TRAX,Pakistan,Logistics Supply Chain Specialist \nCompany Na...,11 mos 10 mos 8 mos 3 yrs 1 yr,NaN,Karachi University APTECH Computer Education g...,"Bachelor of Commerce (BCom) Business/Commerce,...",Teamwork Microsoft Excel Sales Marketing Sales...
257,257,Ahsan Shafiq,Sr. UI Engineer at Folio3 Software,"Karachi Division, Pakistan",NaN,No Expiration Date No Expiration Date,NaN,Muhammad Ali Jinnah University Aptech Computer...,Software Engineering Information Technology BS...,jQuery CSS JavaScript Web Development Mobile A...
258,258,Hammad Masood,CEO| Co-founder at Doozie Labs,Pakistan,Co-founder Software Engineer,8 yrs 2 mos 1 yr 1 mo,NaN,National University of Computer and Emerging S...,Master of Science (MS) Computer Science 3.11 C...,MySQL Android C++ Android Development OOP Prog...
